# Proof of concept for the solving of the transportation problem

This Poc aims at exploring the transportation problem in the given context and try using osr-tools's vrptw model to solve it.

First lets import our data

In [1]:
import pandas as pd

# Load the data from Excel
excel_path = 'Modacruising.xlsx'
meetings_df = pd.read_excel(excel_path)

# Display the first few rows of the dataframe to understand its structure
meetings_df.head()

,Marque,Adresse,Date Ouverture (MM/DD/AAAA),Date Fermeture,Opening hour,Closing hour,Length
0,schouler,"15 Avenue Hoche,75008",2024-06-18,2024-06-23,09:00:00,19:00:00,30.0
1,Proenza,"15 Avenue Hoche,75008",2024-06-18,2024-06-23,09:00:00,19:00:00,30.0
2,retrofete,"15 Avenue Hoche,75008",2024-06-18,2024-06-23,09:00:00,19:00:00,30.0
3,Hemant & Nandita,"15 Avenue Hoche,75008",2024-06-18,2024-06-23,09:00:00,19:00:00,30.0
4,Ser.Ho.Ya,"15 Avenue Hoche,75008",2024-06-18,2024-06-23,09:00:00,16:00:00,30.0


In [2]:
meetings_df[meetings_df['Adresse'].isna()]

,Marque,Adresse,Date Ouverture (MM/DD/AAAA),Date Fermeture,Opening hour,Closing hour,Length
54,Casablanca,NaN,NaT,NaT,NaN,NaN,NaN


We can see that we have a single row with no data so we'll get rid of it.
I also spotted an adress in Saint-tropez wich is not Paris. 
I decided to remove it aswell to simplify the problem in a first part

In [3]:
meetings_df = meetings_df.drop(46).drop(54).reset_index()

In [4]:
import numpy as np

# Extract relevant data
locations = meetings_df['Adresse'].tolist()
brands = meetings_df['Marque'].tolist()
num_locations = len(set(locations))

Lets check length column

In [5]:
meetings_df[meetings_df['Length'].isna()]

,index,Marque,Adresse,Date Ouverture (MM/DD/AAAA),Date Fermeture,Opening hour,Closing hour,Length
52,53,Guidi,"40, Rue Vola 75003",2024-06-20,2024-06-20,16:00:00,16:30:00,NaN


We have only one so we will assume it's 30 minutes like the rest and cast the column to in for the solver


In [6]:
meetings_df['Length'] = meetings_df['Length'].astype('Int64').tolist()
meetings_df.iloc[52, 7] = 30

Here I will index all my adresses to make it easier to retrieve the travel time for each one

In [7]:
meetings_df['Adresse'] = meetings_df['Adresse'].str.upper() + ' Paris, France'
locations = meetings_df['Adresse'].tolist()
locations_uniq = list(set(locations))
meetings_df['AdresseIndex'] = meetings_df.apply(lambda row : locations_uniq.index(row['Adresse']), axis=1)
meetings_df

,index,Marque,Adresse,Date Ouverture (MM/DD/AAAA),Date Fermeture,Opening hour,Closing hour,Length,AdresseIndex
0,0,schouler,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,19:00:00,30,20
1,1,Proenza,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,19:00:00,30,20
2,2,retrofete,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,19:00:00,30,20
3,3,Hemant & Nandita,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,19:00:00,30,20
4,4,Ser.Ho.Ya,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,16:00:00,30,20
5,5,247 Showroom,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,20:00:00,30,20
6,6,Mint,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,19:00:00,30,20
7,7,The Known Agency,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,19:00:00,30,20
8,8,Nahmias,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,19:00:00,30,20
9,9,Willy Chavarria,"15 AVENUE HOCHE,75008 Paris, France",2024-06-18,2024-06-23,09:00:00,19:00:00,30,20


Now the heart of the problem, implementing the time windows correctly. In our data the time windows for a single node are disjointed wich our model will not be able to handle properly. I looked around and found this github post https://github.com/google/or-tools/issues/456 where they propose a solution to this "multiple soft time windows" problem.
Our next steps are:
- Casting our timestamp to minutes and add the hour component of each
- Offseting our dates realtive to the planning horizon (the start of the first meeting)
- Calculating for each node the total span between first opening and last day closing aswell as the time windows where the store is close during this period. This is essential for our model

In [8]:

opening_dates = pd.to_datetime(meetings_df['Date Ouverture (MM/DD/AAAA)'])
closing_dates = pd.to_datetime(meetings_df['Date Fermeture'])
opening_hours = pd.to_datetime(meetings_df['Opening hour'], format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(meetings_df['Opening hour'], format='%H:%M:%S').dt.minute
closing_hours = pd.to_datetime(meetings_df['Closing hour'], format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(meetings_df['Closing hour'], format='%H:%M:%S').dt.minute


In [9]:
# Convert time to minutes from midnight
from datetime import datetime, timedelta
def time_to_minutes(t):
    return t.hour * 60 + t.minute

time_windows = []
start_date = opening_dates.min()

for i in range(len(meetings_df)):
    forbidden_time_frame_starts = []
    forbidden_time_frame_ends = []
    day_windows = []
    current_date = opening_dates[i]
    end_date = closing_dates[i]
    while current_date <= end_date:
        #Calc each starting and ending of period where the shops is closed (at night)
        forbidden_time_frame_starts.append(int(end_time + (current_date - start_date).days * 24 * 60 - 540))
        forbidden_time_frame_ends.append(int(start_time + (current_date - start_date).days * 24 * 60 - 540))

        #Calc the total duration between first opening and last day closing
        start_time = int((opening_dates[i] - start_date).days * 24 * 60 + opening_hours[i] - 540)
        end_time = int((closing_dates[i] - start_date).days * 24 * 60 + closing_hours[i] - 540)

        current_date += timedelta(days=1)
    
    forbidden_time_frame_starts.pop()
    forbidden_time_frame_ends.pop(0)
    time_windows.append((start_time, end_time, forbidden_time_frame_starts, forbidden_time_frame_ends, meetings_df.iloc[i]['AdresseIndex']))

[(0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7620, [420, 1860, 3300, 4740, 6180], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7860, [660, 2100, 3540, 4980, 6420], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7800, [600, 2040, 3480, 4920, 6360], [1440, 2880, 4320, 5760, 7200], 20),
 (0, 7680, [480, 1920, 3360, 4800, 6240], [1440, 288

Now we need to build a matrix that allows us to know the travel time between each nodes.
For that we can just get every unique point and calculate the distance with every others.

- Lets get rid of the duplicates taking in account the casing.
- We will want to convert these postal adresses to coordinates using OSR:
  
      Since the adresses in the dataset do not specify that the city is Paris in some cases, OSR api can pickup streets with the same names in other cities 
      We can adjust for that by adding "Paris, France" to every adresses. We also upper case every string for homogeneity
    

- Then we can use these coordinates to get the estimated travel time in a car, again with OSR

In [10]:
api_key_ors = '<KEY>'

In [11]:
import requests

def geocode_address(address, api_key):
    url = f'https://api.openrouteservice.org/geocode/search'
    params = {
        'api_key': api_key,
        'text': address,
        'size': 1
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json()['features']
        if results:
            return results[0]['geometry']['coordinates'][1], results[0]['geometry']['coordinates'][0]
    return None

print(locations_uniq)

['194 RUE DE RIVOLI 75001 Paris, France', "14 RUE DU CHATEAU D'EAU,75010 Paris, France", '5 RUE SAINTE-ANASTASE,75003 Paris, France', '2 PLACE SKANDERBERG 75019  Paris, France', '14 RUE DE BRETAGNE 3RD FLOOR, 75003 Paris, France', '5 RUE CHAPON 75003 Paris, France', '4 RUE DE JARENTE 75004 Paris, France', '137 RUE DU FAUBOURG SAINT-HONORE 75008 Paris, France', '5 RUE DU GRENIER-SAINT-LAZARE 75003 Paris, France', '58 RUE CHARLOT,75003 Paris, France', '236 RUE SAINT MARTIN,75003 Paris, France', '112 RUE DE RICHELIEU,75002 Paris, France', '1 IMPASSE SAINT CLAUDE 75003 Paris, France', "6 RUE DE L'AMIRAL DE COLIGNY,75001 Paris, France", '19 RUE DES MINIMES,75003 Paris, France', "15 RUE DE L'ECOLE DE MÉDECINE,75006 Paris, France", '241 RUE SAINT-MARTIN,75003 Paris, France', '44 RUE DE SEVIGNE 75003 Paris, France', '11 AVENUE IENA 75016 Paris, France', '4 RUE DE GALLIERA,75116 Paris, France', '15 AVENUE HOCHE,75008 Paris, France', '118 RUE DE TURENNE,75003 Paris, France', '6 RUE DES COUTURES 

In [12]:
## RETURNED DATA: 
# coordinates = [(48.862997, 2.35331), (48.860592, 2.359476), (48.858705, 2.342865), (48.865618, 2.374666), (48.841109, 2.338748), (48.860405, 2.357272), (48.858705, 2.342865), (48.869165, 2.362081), (48.863765, 2.33221), (48.865754, 2.297525), (48.868893, 2.355487), (48.856696, 2.362713), (48.862535, 2.365644), (48.850428, 2.340801), (48.862983, 2.357237), (48.860136, 2.362274), (48.855702, 2.362962), (48.865577, 2.305426), (48.857303, 2.36502), (48.859523, 2.364027), (48.868475, 2.32553), (48.859951, 2.340349), (48.868893, 2.355487), (48.861083, 2.361731), (48.86359, 2.355464), (48.863199, 2.361914), (48.866821, 2.342605), (48.860335, 2.365678), (48.866112, 2.327788), (48.858705, 2.342865), (48.863403, 2.289233), (48.862549, 2.362179), (48.869218, 2.320194), (48.858705, 2.342865), (48.865699, 2.336738), (48.877764, 2.304577), (48.863507, 2.364778), (48.865213, 2.36298), (48.87577, 2.291876), (48.859487, 2.364536), (48.860882, 2.361234), (48.865255, 2.34491)]

In [13]:
import requests


def get_trave_time_matrix(coordinates, api_key):
    url = 'https://api.openrouteservice.org/v2/matrix/driving-car'
        
    # Préparation des coordonnées pour la requête
    locations_for_api = [[lon, lat] for lat, lon in coordinates]
    
    params = {
        'locations': locations_for_api,
        'metrics': ['duration']
    }
    
    response = requests.post(url, json=params, headers={'Authorization': api_key})

    if response.status_code == 200:
        data = response.json()
        return data['durations']
    else:
        print(f'Erreur: {response.status_code}, {response.text}')
        return None

# travel_times_matrix = get_trave_time_matrix(coordinates, api_key_ors)

In [14]:
df = pd.DataFrame(travel_times_matrix) / 60
df.to_csv('travel_matrix.csv', index=False)

df.columns = locations_uniq
df.index = locations_uniq

In [15]:
df = df + 30 #To represent the 30 mn of each meetings
df

,"194 RUE DE RIVOLI 75001 Paris, France","14 RUE DU CHATEAU D'EAU,75010 Paris, France","5 RUE SAINTE-ANASTASE,75003 Paris, France","2 PLACE SKANDERBERG 75019 Paris, France","14 RUE DE BRETAGNE 3RD FLOOR, 75003 Paris, France","5 RUE CHAPON 75003 Paris, France","4 RUE DE JARENTE 75004 Paris, France","137 RUE DU FAUBOURG SAINT-HONORE 75008 Paris, France","5 RUE DU GRENIER-SAINT-LAZARE 75003 Paris, France","58 RUE CHARLOT,75003 Paris, France",...,"17 RUE CHAPON 75003 Paris, France","21 RUE DES FILLES DU CALVAIRE 75003 Paris, France","40, RUE VOLA 75003 Paris, France","14 RUE DE LA CORDERIE,75003 Paris, France","5 RUE DE CASTIGLIONE,75001 Paris, France","13 AVENUE DU PRESIDENT WILSON,75016 Paris, France","SALLE ERARD, 13 RUE DU MAIL 75002 Paris, France","132 RUE DE TURENNE 75003 Paris, France","7 RUE DUPHOT 75001 Paris, France","18 RUE JEAN GOUJON 75008 Paris, France"
"194 RUE DE RIVOLI 75001 Paris, France",30.000000,32.114667,35.065667,37.096500,40.784500,31.370667,35.065667,35.713667,35.508833,45.462500,...,40.827833,35.065667,34.408000,43.336667,33.699667,34.053500,45.716333,33.627167,32.876833,32.151000
"14 RUE DU CHATEAU D'EAU,75010 Paris, France",37.013833,30.000000,40.875667,35.712333,46.117333,38.016333,40.875667,36.494833,41.318833,49.767833,...,45.133333,40.875667,41.090000,47.642167,32.330833,35.043000,50.021667,32.318500,30.762167,37.961000
"5 RUE SAINTE-ANASTASE,75003 Paris, France",33.868000,35.982667,30.000000,39.998500,39.792500,35.238667,30.000000,38.577000,35.212500,44.872500,...,41.111333,30.000000,38.043333,43.954333,37.278333,36.928333,46.334000,37.225833,36.744833,34.905667
"2 PLACE SKANDERBERG 75019 Paris, France",36.895333,38.642000,40.757167,30.000000,47.143667,37.897833,40.757167,36.376500,41.200333,49.649500,...,45.014833,40.757167,40.971667,47.523667,35.494833,37.540833,49.903333,35.469333,37.258333,37.842500
"14 RUE DE BRETAGNE 3RD FLOOR, 75003 Paris, France",38.762167,40.876833,37.698833,44.892667,30.000000,40.132833,37.698833,43.471167,40.482000,45.375333,...,45.782500,37.698833,43.278833,48.625667,42.172333,41.822500,48.986167,42.120000,41.639000,39.908500
"5 RUE CHAPON 75003 Paris, France",31.847833,30.744167,36.913333,35.726000,42.264167,30.000000,36.913333,34.959500,37.356500,47.310167,...,42.675667,36.913333,36.255833,45.184500,32.329167,32.682833,47.564167,32.574167,31.506333,33.998833
"4 RUE DE JARENTE 75004 Paris, France",33.868000,35.982667,30.000000,39.998500,39.792500,35.238667,30.000000,38.577000,35.212500,44.872500,...,41.111333,30.000000,38.043333,43.954333,37.278333,36.928333,46.334000,37.225833,36.744833,34.905667
"137 RUE DU FAUBOURG SAINT-HONORE 75008 Paris, France",33.968000,35.714500,37.829833,34.506333,44.384500,34.970500,37.829833,30.000000,38.273000,45.906833,...,41.272333,37.829833,37.229000,43.781167,33.761167,34.613500,46.160833,34.538667,36.327667,34.915167
"5 RUE DU GRENIER-SAINT-LAZARE 75003 Paris, France",35.152000,37.266667,33.709500,41.245667,43.491167,36.522500,33.709500,39.592167,30.000000,41.061167,...,36.092333,33.709500,33.588667,38.935500,38.525500,38.175667,41.315000,38.770500,38.028833,33.436167
"58 RUE CHARLOT,75003 Paris, France",43.732333,45.847000,40.144833,49.862667,45.591333,45.102833,40.144833,48.318667,39.821333,30.000000,...,38.274500,40.144833,42.315333,35.570000,47.142500,46.792667,35.546500,47.359500,46.609167,42.016500


In [16]:
time_windows.pop(15) #Removing because it is heavy on the solution calcul

(42060, 42090, [], [], 36)

In [26]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Create model
manager = pywrapcp.RoutingIndexManager(len(time_windows[:15]), 1, 0)
routing = pywrapcp.RoutingModel(manager)
routing.routing_solution_limit = 1
solver = routing.solver()

# Define cost function (travel time)
def travel_time_callback(from_index, to_index):
    return int(df.iloc[meetings_df.iloc[from_index]['AdresseIndex'], meetings_df.iloc[to_index-1]['AdresseIndex']])

# transit_callback_index = routing.RegisterTransitMatrix(list(df.astype(int).itertuples(index=False)))
transit_callback_index = routing.RegisterTransitCallback(travel_time_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

routing.AddDimension(
    transit_callback_index,
    30, 
    1000, 
    False,
    'Time'
)

time_dimension = routing.GetDimensionOrDie('Time')

# Time windows constraints
for idx, day_windows in enumerate(time_windows[:15]):
    (start, end, forbs_starts, forbs_ends, _) = day_windows
    index = manager.NodeToIndex(idx)
    if start < end:
        print(f"{index} is {start} to {end} fs {forbs_starts} fe{forbs_ends}")
        time_dimension.CumulVar(index).SetRange(start, end)

        if len(forbs_starts) > 0 and len(forbs_ends) > 0:
            solver.AddConstraint(
                solver.NotMemberCt(
                    time_dimension.CumulVar(index), 
                    forbs_starts, forbs_ends
                )
            )
    else:
        print(f"Invalid time window for meeting {idx} at {index}")

# # Allow waiting time at the nodes
# for node in range(len(time_windows)):
#     index = manager.NodeToIndex(node)
#     routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(index))

# Set the search parameters
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

solution = routing.SolveWithParameters(search_parameters)

0 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
1 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
2 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
3 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
4 is 0 to 7620 fs [420, 1860, 3300, 4740, 6180] fe[1440, 2880, 4320, 5760, 7200]
5 is 0 to 7860 fs [660, 2100, 3540, 4980, 6420] fe[1440, 2880, 4320, 5760, 7200]
6 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
7 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
8 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
9 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
10 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
11 is 0 to 7800 fs [600, 2040, 3480, 4920, 6360] fe[1440, 2880, 4320, 5760, 7200]
12 is 0 to 7680 fs [480, 1

In [27]:
# Print the solution 
time_dimension = routing.GetDimensionOrDie("Time")
if solution:
    print('Objective: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    while not routing.IsEnd(index):
        time_var = time_dimension.CumulVar(index)
        node_index = manager.IndexToNode(index)
        print(f'Meeting at {locations[node_index]} for brand {brands[node_index]} timemin: {solution.Min(time_var)} timemax: {solution.Max(time_var)} ')
        index = solution.Value(routing.NextVar(index))
else:
    print('No solution found!')

Objective: 450
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand schouler  timemin: 0 timemax: 149 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand Cinq à Sept  timemin: 30 timemax: 179 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand Atelier timemin: 60 timemax: 209 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand Veronica Beard timemin: 90 timemax: 239 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand Willy Chavarria timemin: 120 timemax: 269 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand Nahmias timemin: 150 timemax: 299 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand The Known Agency timemin: 180 timemax: 329 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand Mint timemin: 210 timemax: 359 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand 247 Showroom timemin: 240 timemax: 389 
Meeting at 15 AVENUE HOCHE,75008 Paris, France for brand Ser.Ho.Ya timemin: 270 timemax: 419 
Meeting at 15 AVENUE HOCHE,75008 Pa

# Conclusions

- The vrptw model of the or-tools library can be used to solve our problems but several optimization would need to be made.
First, because I'm using python, there is alot of boiler plate processing going on and slowing the process. We could implement this solution https://groups.google.com/g/or-tools-discuss/c/ViEPGrYBils to mitigate the effect. Right now it is not performant inoff for me to go through all the dataset with the model. It seems especially true for dates further apart. 

- Regarding the adaptability of this solution, since I used Open Route Service wich is free and easy to use, it is not taking the traffic in account. This can be solved by simply switching the API for the travel time matrix retrieval (with google maps's for example) and update the travel time matrix, or part of it, regularily.

- My implementation fails to properly incorporate the travel time in the results. This is probably due to issue with my hyperparameters
  
- For the reaction to external events, cancellation etc... We could just recalculate the different itineraries taking in account for the unexpected change by removing it from the list